In [1]:
!pip install plotly[express]
!pip install kaggle

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px 
import matplotlib.pyplot as plt
import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi
import scipy.stats as stats
from scipy.stats import shapiro

In [3]:
%run "../src/descarga remota.py"

Iniciando descarga del dataset: titanic
Descarga completada. Archivos guardados en: ../data
Descomprimiendo titanic.zip...
Archivo titanic.zip descomprimido y eliminado.

¡Proceso de descarga y extracción finalizado con éxito!


In [4]:
df_test = pd.read_csv("../data/test.csv")
df_train = pd.read_csv("../data/train.csv")

In [5]:
print("DataFrame de test.csv: ")
display(df_test.head(5))

print("DataFrame de train.csv: ")
display(df_train.head(5))

DataFrame de test.csv: 


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


DataFrame de train.csv: 


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Descripción variables 
## (que representa cada una y un breve analisis estadistico basico)

Basta con observar el apartado **Data Dictionary** en la pagina de [kaggle](https://www.kaggle.com/competitions/titanic/data) y es posible observar el significado de cada variable.

## Descripción de Variables del Conjunto de Datos

| Variable | Definición | Key (Clave) |
| :--- | :--- | :--- |
| **survival** | Supervivencia | 0 = No, 1 = Sí |
| **pclass** | Clase del billete/boleto | 1 = 1ra, 2 = 2da, 3 = 3ra |
| **sex** | Sexo | |
| **Age** | Edad en años | |
| **sibsp** | # de hermanos / cónyuges a bordo del Titanic | |
| **parch** | # de padres / niños a bordo del Titanic | |
| **ticket** | Número del billete/boleto | |
| **fare** | Tarifa del pasajero | |
| **cabin** | Número de cabina | |
| **embarked** | Puerto de Embarque | C = Cherbourg, Q = Queenstown, S = Southampton |

#### Ademas se hacen otras salvedades sobre las variables

##### **pclass:** Un proxy para el estatus socioeconómico (SES)
* **1ra:** Alta (Upper)
* **2da:** Media (Middle)
* **3ra:** Baja (Lower)

---

##### **age:** Edad
* La edad es **fraccionaria** si es menor de 1 año.
* Si la edad es **estimada**, se presenta en la forma de *xx.5* (ej. 29.5).

---

##### **sibsp:** Relaciones familiares (Hermanos/Cónyuges)
El conjunto de datos define las relaciones familiares de esta manera...
* **Hermano/a:** hermano, hermana, hermanastro, hermanastra
* **Cónyuge:** marido, mujer
    > *(Se ignoraron a las amantes y prometidos/as.)*

---

##### **parch:** Relaciones familiares (Padres/Hijos)
El conjunto de datos define las relaciones familiares de esta manera...
* **Padre:** madre, padre
* **Hijo/a:** hija, hijo, hijastra, hijastro
    > **Nota:** Algunos niños viajaron solo con una niñera, por lo tanto, **parch=0** para ellos.

> Note que la variable **Survived** solo esta en **train**, **no** en **test** ya que es la **target variable**

### Información Basica de los DataFrame

In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


> Aprecie que **Cabin** tiene tan solo 91 datos no nulos, un problema con el que se lideara mas adelante.

> hay 4  variables int, 5 object y 2 float

### Limpieza de datos

In [7]:
df_train.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

In [40]:

total_duplicados_train = df_train.duplicated().sum()
total_duplicados_test = df_test.duplicated().sum()
print(f"Número total de filas duplicadas en train: {total_duplicados_train}")
print(f"Número total de filas duplicadas en test: {total_duplicados_test}")

Número total de filas duplicadas en train: 0
Número total de filas duplicadas en test: 0


> No hay duplicados

#### Limpieza df_train

Note que las columnas del Data Frame **df_train** las cuales son **Age**, **Cabin**, **Embarked**; poseen datos nulos, se procederá a analizar la mejor estrategia para lidiar con los valores nulos.

##### Limpieza columna **Age**

In [8]:
df_train["Age"]

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888     NaN
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

Se revisa cuantos datos nulos existen en la columna **Age** y se contrasta con el total de los datos.

In [9]:
df_train["Age"].isnull().sum()

np.int64(177)

In [10]:
len(df_train)

891

Se revisa cuanto porcentaje de datos nulos existe.

In [11]:
porcentaje_nulos = df_train["Age"].isnull().mean() * 100
print(f"El porcentaje de nulos en la columna'Age' es: {porcentaje_nulos:.2f}%")

El porcentaje de nulos en la columna'Age' es: 19.87%


Luego de esto, evaluamos la normalidad de la variable **Age**, para esto creamos el *Histograma* de la columna, además se evalúan la similitud entre las medidas de tendencia central.

In [12]:

fig = px.histogram(
    df_train,
    x='Age',
    nbins=30,
    title='Distribución de Edad (Age) en df_train',
    marginal='box',
    color_discrete_sequence=["#0052CC"]
)

fig.update_layout(
    xaxis_title='Edad',
    yaxis_title='Frecuencia (Conteo)',
    bargap=0.05,
    height=500
)


fig.show()

En la gráfica se puede observar un sesgo hacia la derecha, lo que indica falta de normalidad en la variable.

In [13]:
stat, p = shapiro(df_train['Age'].dropna())
print(f"Shapiro-Wilk: stat={format(stat, '.6f')}, p-value={format(p, '.10f')}")

Shapiro-Wilk: stat=0.981458, p-value=0.0000000734


Dado que p es menor que 0.05 se descarta la hipótesis de normalidad. Es decir, los datos no siguen una distribución normal.

Se decide escoger la mediana como valores para rellenar los valores de la columna **Age**.

In [14]:
df_train["Age"]=df_train["Age"].fillna(df_train["Age"].median())
df_train["Age"].isnull().sum()

np.int64(0)

Note que ahora la columna está totalmente limpia.

##### Creación de función para sistematizar la limpieza de variables numéricas.

In [15]:
#Modo de uso:
#Pase la base de datos y columna de esta, se redefine la columna en base a la serie que devuelve la función
def limpieza_datos_numericos(df,col):
    stat, p = shapiro(df[col].dropna())
    if p<=0.05:
        print("La variable no sigue una distribución normal.")
        print("Se completan los datos nulos con la mediana.")
        a=df[col].fillna(df[col].median())
        return  a
    else:
        print("La variable sigue una distribución normal.")
        print("Se completan los datos nulos con la media.")
        a=df[col].fillna(df[col].mean())
        return a

##### Limpieza columna **Cabin**

In [16]:
df_train["Cabin"]

0       NaN
1       C85
2       NaN
3      C123
4       NaN
       ... 
886     NaN
887     B42
888     NaN
889    C148
890     NaN
Name: Cabin, Length: 891, dtype: object

Se revisa el número total de datos nulos en la columna **Cabin**

In [17]:
df_train["Cabin"].isnull().sum()

np.int64(687)

Como es una variable categorica, se completan los datos nulos con "Desconocido", ya que no podemos intuir nada con datos numéricos.

In [18]:
df_train["Cabin"]=df_train["Cabin"].fillna("Desconocido")
df_train["Cabin"]

0      Desconocido
1              C85
2      Desconocido
3             C123
4      Desconocido
          ...     
886    Desconocido
887            B42
888    Desconocido
889           C148
890    Desconocido
Name: Cabin, Length: 891, dtype: object

Note que ahora la columna está totalmente limpia.

##### Limpieza columna **Embarked**

In [19]:
df_train["Embarked"]

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

Se mira cuantos datos nulos hay en la columna **Embarked**.

In [20]:
df_train["Embarked"].isnull().sum()

np.int64(2)

Optamos por eliminar los datos, ya que son poco relevantes para la densidad que tenemos.

In [21]:
df_train=df_train.dropna(subset=["Embarked"])
df_train["Embarked"].isnull().sum()

np.int64(0)

Note que ahora la columna está totalmente limpia.

#### Limpieza df_test

In [22]:
df_test.isnull().any()

PassengerId    False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare            True
Cabin           True
Embarked       False
dtype: bool

Note que las columnas del Data Frame **df_test** las cuales son **Age**, **Fare**, **Cabin**; poseen datos nulos, se procederá a analizar la mejor estrategia para lidiar con los valores nulos.

##### Limpieza columna **Age**

In [23]:
df_test["Age"]

0      34.5
1      47.0
2      62.0
3      27.0
4      22.0
       ... 
413     NaN
414    39.0
415    38.5
416     NaN
417     NaN
Name: Age, Length: 418, dtype: float64

Miramos cuantos datos nulos hay en la columna **Age**

In [24]:
len(df_test)

418

In [25]:
df_test["Age"].isnull().sum()

np.int64(86)

Se revisa cuanto porcentaje de datos nulos existe.

In [26]:
porcentaje_nulos = df_test["Age"].isnull().mean() * 100
print(f"El porcentaje de nulos en la columna'Age' es: {porcentaje_nulos:.2f}%")

El porcentaje de nulos en la columna'Age' es: 20.57%


Utilizamos la función normalizada para la limpieza de datos numéricos

In [27]:
columna_limpia=limpieza_datos_numericos(df_test,"Age")
df_test["Age"]=columna_limpia

La variable no sigue una distribución normal.
Se completan los datos nulos con la mediana.


In [28]:
df_test["Age"].isnull().sum()

np.int64(0)

Note que ahora la columna está totalmente limpia.

##### Limpieza de la columna **Fare**

In [29]:
df_test["Fare"]

0        7.8292
1        7.0000
2        9.6875
3        8.6625
4       12.2875
         ...   
413      8.0500
414    108.9000
415      7.2500
416      8.0500
417     22.3583
Name: Fare, Length: 418, dtype: float64

Se revisan la cantidad de datos nulos en la columna **Fare**

In [30]:
df_test["Fare"].isnull().sum()

np.int64(1)

Optamos por eliminar el dato, ya que es poco relevante para la densidad que tenemos.

In [31]:
df_test=df_test.dropna(subset=["Fare"])
df_test["Fare"].isnull().sum()

np.int64(0)

Note que ahora la columna está totalmente limpia.

##### Limpieza columna **Cabin**

In [32]:
df_test["Cabin"]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
413     NaN
414    C105
415     NaN
416     NaN
417     NaN
Name: Cabin, Length: 417, dtype: object

Se revisa el número total de datos nulos en la columna **Cabin**

In [33]:
df_test["Cabin"].isnull().sum()

np.int64(326)

Como es una variable categorica, se completan los datos nulos con "Desconocido", ya que no podemos intuir nada con datos numéricos.

In [34]:
df_test["Cabin"]=df_test["Cabin"].fillna("Desconocido")
df_test["Cabin"]

0      Desconocido
1      Desconocido
2      Desconocido
3      Desconocido
4      Desconocido
          ...     
413    Desconocido
414           C105
415    Desconocido
416    Desconocido
417    Desconocido
Name: Cabin, Length: 417, dtype: object

Note que ahora la columna está totalmente limpia.

### Analisis estadistico basico para las variables:

In [35]:
columnas = df_train.columns   #se usa el de todas las col

In [44]:
def analizar(data1, data2, cols,nombre_base1=None, nombre_base2=None):
    for col in cols:
        try:
            desc1=data1[col].describe()
            desc2=data2[col].describe()
            if nombre_base1 and nombre_base2:
                resultado_final = pd.concat([desc1, desc2], axis=1, keys=[nombre_base1, nombre_base2])
            else:
                resultado_final = pd.concat([desc1, desc2], axis=1, keys=['DF_A', 'DF_B'])
            print(f"columna: {col}")
            print(resultado_final)
            print("\n\n\n")
        except Exception as e:
            print("columna no esta\n\n\n")

#### analisis general y comparación entre train y test

In [45]:
# analisis general para df_test
analizar(df_test, df_train, columnas, nombre_base1="test", nombre_base2="train")

columna: PassengerId
              test       train
count   417.000000  889.000000
mean   1100.635492  446.000000
std     120.923774  256.998173
min     892.000000    1.000000
25%     996.000000  224.000000
50%    1101.000000  446.000000
75%    1205.000000  668.000000
max    1309.000000  891.000000




columna no esta



columna: Pclass
             test       train
count  417.000000  889.000000
mean     2.263789    2.311586
std      0.842077    0.834700
min      1.000000    1.000000
25%      1.000000    2.000000
50%      3.000000    3.000000
75%      3.000000    3.000000
max      3.000000    3.000000




columna: Name
                            test                train
count                        417                  889
unique                       417                  889
top     Peter, Master. Michael J  Dooley, Mr. Patrick
freq                           1                    1




columna: Sex
        test train
count    417   889
unique     2     2
top     male  male
freq     2